In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import missingno as mn
import seaborn as sb
from scipy.stats import chi2_contingency
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split



#Creacion del modelo con RLM
#modelo predictivo que ayuda a entender la demanda de bicicletas en distintas circunstancias 

#PRIMER MODELO
#variables a incluir que influyen de acuerdo al analisis exploratorio de los datos
#-Hora del dia, temperatura, visibilidad, radiacion solar, estacion, dia festivo.



#Importar datos desde el archivo de CSV
archivo = "C:/Users/natal/OneDrive/Documentos/U/2024-2/Analitica/PROYECTO1/datoslimpios.csv"
datos = pd.read_csv(archivo)
datos.columns


Index(['Date', 'Rented Bike Count', 'Hour', 'Temperature(C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)',
       'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter',
       'Holiday_No Holiday'],
      dtype='object')

In [2]:
#modelo con division de datos pata entrenamiento 
## Valores x y y,
from sklearn.linear_model import LinearRegression

X = datos.loc[:, ['Hour', 'Temperature(C)','Visibility (10m)','Solar Radiation (MJ/m2)',
        'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter','Holiday_No Holiday']]

y = datos.loc[:, ['Rented Bike Count']]


# Creamos conjuntos de datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Entrenamos el modelo
lr.fit(X_train, y_train)

# Predecimos usando los datos de prueba
y_pred = lr.predict(X_test)

# Puntaje del modelo (coeficiente de determinación R^2)
puntaje = lr.score(X_test, y_test)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_[0]  # Convierte el intercepto a un valor escalar

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes:")
print(coef_df)
print(f"Intercepto: {intercepto}")
print('\n')
# Mostrar el puntaje del modelo
print(f"coeficiente de determinación R^2: {puntaje}")

Coeficientes:
                         Coeficiente
Hour                       31.434482
Temperature(C)             25.073896
Visibility (10m)            0.166016
Solar Radiation (MJ/m2)    18.215486
Seasons_Spring           -112.249882
Seasons_Summer           -225.359614
Seasons_Winter           -266.505786
Holiday_No Holiday        118.077452
Intercepto: -160.4765422592078


coeficiente de determinación R^2: 0.5090449086255487


In [3]:
from sklearn.model_selection import cross_val_score


#modelo con  validacion cruzada 

# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Visibility (10m)','Solar Radiation (MJ/m2)',
        'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter','Holiday_No Holiday']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  # cv=5 indica 5 particiones



# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final con visibility: {r2_final}")


Coeficientes del modelo final:
                         Coeficiente
Hour                       32.283928
Temperature(C)             24.708374
Visibility (10m)            0.166181
Solar Radiation (MJ/m2)    18.715573
Seasons_Spring           -109.250634
Seasons_Summer           -202.738891
Seasons_Winter           -265.781778
Holiday_No Holiday        118.666026
Intercepto del modelo final: -171.16490982475295
Coeficiente de determinación R² del modelo final con visibility: 0.4956349074560348


In [4]:
#modelo 2 validacion cruzada sin visibility
# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Solar Radiation (MJ/m2)',
        'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter','Holiday_No Holiday']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  

# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final sin visibility: {r2_final}")


Coeficientes del modelo final:
                         Coeficiente
Hour                       33.754583
Temperature(C)             23.024870
Solar Radiation (MJ/m2)    41.109020
Seasons_Spring           -166.625602
Seasons_Summer           -195.624792
Seasons_Winter           -307.324630
Holiday_No Holiday        107.769515
Intercepto del modelo final: 92.734047258121
Coeficiente de determinación R² del modelo final sin visibility: 0.47280559616146345


In [6]:
#modelo 3 validacion cruzada sin visibility y sin dummies holiday
# Valores X y Y
X = datos.loc[:,['Hour', 'Temperature(C)','Solar Radiation (MJ/m2)',
        'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  # cv=5 indica 5 particiones



# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final sin visibility y sin holiday: {r2_final}")

Coeficientes del modelo final:
                         Coeficiente
Hour                       33.824115
Temperature(C)             22.685907
Solar Radiation (MJ/m2)    41.327449
Seasons_Spring           -165.180150
Seasons_Summer           -188.350837
Seasons_Winter           -317.057920
Intercepto del modelo final: 198.9314715923831
Coeficiente de determinación R² del modelo final sin visibility y sin holiday: 0.4715418091652147


In [7]:
#modelo 4 (final) con  validacion cruzada 

# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Visibility (10m)','Solar Radiation (MJ/m2)',
        'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  # cv=5 indica 5 particiones


# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final con visibility sin holiday: {r2_final}")

Coeficientes del modelo final:
                         Coeficiente
Hour                       32.369505
Temperature(C)             24.324980
Visibility (10m)            0.165157
Solar Radiation (MJ/m2)    19.093888
Seasons_Spring           -108.013345
Seasons_Summer           -194.690181
Seasons_Winter           -276.749024
Intercepto del modelo final: -52.670537013078615
Coeficiente de determinación R² del modelo final con visibility sin holiday: 0.49410350579908313
